---
title: estimate_fissure_length
description: ''
date: '2022-10-18T21:08:07.863Z'
name: estimate-fissure-length
oxa: oxa:1Bk7uPlcMuaTyKEshESj/DXSdGGDSU9Q2Jlc3twNR
short_title: ''
tags: []
---

## Estimating Fissue Length

Using mapping data from the Compernicus Emergency Response service. Vector layers taken from [here](https://emergency.copernicus.eu/mapping/list-of-components/EMSR546) and loaded as GeoJson

In [1]:
import glob
import zipfile
import re
from os import path


files = glob.glob('../vectors/*.zip')
items = []
for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('shapes/')
    result = re.search(r'EMSR546_AOI01_GRA_MONIT(\d+)_', file)
    result.group(1)
    items.append(dict(index=int(result.group(1)), name=path.basename(file)[:-4]))
    
items = sorted(items, key=lambda x: x['index'])
items[0]['name'] = items[0]['name'].replace('MONIT00', 'PRODUCT')
print(f"Loaded {len(items)} packages")

Loaded 63 packages


In [2]:
geojson_files = glob.glob('shapes/*observedEventP*.json')

items = []
for file in geojson_files:
    if re.search('PRODUCT', file):
        items.append(dict(index=0, filename=file))
    else:
        result = re.search(r'EMSR546_AOI01_GRA_MONIT(\d+)_', file)
        result.group(1)
        items.append(dict(index=int(result.group(1)), filename=file))

items = sorted(items, key=lambda x: x['index'])
print(f"Loaded {len(items)} packages")
print(items[0])

Loaded 63 packages
{'index': 0, 'filename': 'shapes/EMSR546_AOI01_GRA_PRODUCT_observedEventP_r1_v1.json'}


In [3]:
import json

def load_geojson(filename: str):
    with open(filename, 'r') as f:
        data = json.load(f)
    with open(filename[:-5]+'.xml', 'r') as xml:
        xml_str = xml.read();
        result = re.search(r'<gco:Date>(.*)</gco:Date>', xml_str)
    data['name'] = result.group(1)
    return data
                
items_json = [dict(geo=load_geojson(item['filename']), **item) for item in items];
dataset = dict(packages=items_json)

In [4]:
with open('alldata.json','w') as f:
    json.dump(dataset,f)

In [9]:
from ipyleaflet import GeoJSON
from ipywidgets import Layout, Label

label = Label(layout=Layout(width="100%"))
label.value = 'Empty'

def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = f"{properties['obj_desc']} {feature['geometry']['coordinates']}"

def to_geojson(data, color='black'):
    layer = GeoJSON(
        name=data['name'],
        data=data,
        style={
            'color':color, 'opacity': 1,'fillOpacity': 0.5, 'weight': 1
        },
        point_style={
            'type':'circle',
            'radius':7
        },
        hover_style={
            'color': 'red', 'dashArray': '0', 'fillOpacity': 0.5
        },
    )
    layer.on_hover(hover_handler)
    return layer

vents_layers = { item['geo']['name']: item['geo'] for item in items_json }

In [23]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl,MeasureControl,FullScreenControl
from ipywidgets import Layout, Label, VBox
import geopandas as gpd

m = Map(center=(28.612915,-17.866048), zoom=16, height=500, layout=Layout(height='800px'))

# m.add_layer(to_geojson(vents_layers['2021-12-18'], 'black'))
# m.add_layer(to_geojson(vents_layers['2021-12-02'], 'green'))
m.add_layer(to_geojson(vents_layers['2021-11-25'], 'blue'))
# m.add_layer(to_geojson(vents_layers['2021-10-11'], 'red'))


measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)

m.add_control(measure)
m.add_control(LayersControl())
m.add_control(FullScreenControl())

m

VBox([m, label])

In [16]:
import numpy as np
import haversine as hs

def dist(x,y):   
    return np.sqrt(np.sum((np.asarray(x)-np.asarray(y))**2))


red_a = (-17.869579, 28.615856)
red_b = (-17.867162, 28.614473)

red_ab = 1000*hs.haversine(red_a, red_b)
print(f"Red a-b {red_ab:.0f}m")


blue_a = [-17.86979, 28.616085]
blue_b = [-17.864527, 28.612911]

blue_ab = 1000*hs.haversine(blue_a, blue_b)
print(f"Blue a-b {blue_ab:.0f}m")

green_a = [-17.871064, 28.616063]
green_b = [-17.864527, 28.612911]
green_ab = 1000*hs.haversine(green_a, green_b)
print(f"Green a-b {green_ab:.0f}m")


black_a = [-17.87063, 28.616106]
black_b = [-17.864119, 28.612807]
black_c = [-17.865459, 28.612676]

black_ab = 1000*hs.haversine(black_a, black_b)
print(f"Black a-b {black_ab:.0f}m")
black_ac = 1000*hs.haversine(black_a, black_c)
print(f"Black a-c {black_ac:.0f}m")


Red a-b 306m
Blue a-b 675m
Green a-b 800m
Black a-b 804m
Black a-c 680m
